In [257]:
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import pandas as pd
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import sys
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [258]:
#read the data
data = pd.read_csv("f7664209-d97b-4f9b-a7bd-e1fef352177c_Data.csv")

print(data.head())

   Time Time Code    Country Name Country Code  \
0  1992    YR1992     Afghanistan          AFG   
1  1992    YR1992         Albania          ALB   
2  1992    YR1992         Algeria          DZA   
3  1992    YR1992  American Samoa          ASM   
4  1992    YR1992         Andorra          AND   

  GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]  \
0                                                 ..                        
1                                   3275.44433583801                        
2                                   8383.77024779121                        
3                                                 ..                        
4                                                 ..                        

  CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]  
0                                 0.0961965810608727       
1                                  0.774724910911141       
2                                   2.96498636282543    

In [259]:
data.columns

Index(['Time', 'Time Code', 'Country Name', 'Country Code',
       'GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]',
       'CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]'],
      dtype='object')

In [260]:
data.drop(['Time Code','Country Code'],axis=1,inplace=True)

In [261]:
cols = {'GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]':'gdppc',
          'CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]':'co2pc',
        'Country Name':'country',
       'Time':'year'}
data.rename(columns=cols, inplace=True)

In [262]:
data.columns

Index(['year', 'country', 'gdppc', 'co2pc'], dtype='object')

In [263]:
data.coyntry=data.country.astype("category")

In [264]:
data.head()

,year,country,gdppc,co2pc
0,1992,Afghanistan,..,0.0961965810608727
1,1992,Albania,3275.44433583801,0.774724910911141
2,1992,Algeria,8383.77024779121,2.96498636282543
3,1992,American Samoa,..,..
4,1992,Andorra,..,6.91205338948512


In [265]:
data.replace('..',np.NaN,inplace=True)

In [266]:
data.isnull().sum()

year         3
country      5
gdppc      848
co2pc      848
dtype: int64

In [267]:
data.groupby('country').agg(lambda x: x.isnull().sum()).sort_values(['gdppc']+['co2pc'],ascending=False).head(60)

,year,gdppc,co2pc
country,,,
American Samoa,0,27,27
Channel Islands,0,27,27
Guam,0,27,27
Isle of Man,0,27,27
Monaco,0,27,27
Northern Mariana Islands,0,27,27
St. Martin (French part),0,27,27
Virgin Islands (U.S.),0,27,27
South Sudan,0,27,22


In [268]:
temp=data.dropna(axis=0,thresh=4)

In [269]:
temp.isnull().sum()

year       0
country    0
gdppc      0
co2pc      0
dtype: int64

In [270]:
temp

,year,country,gdppc,co2pc
1,1992,Albania,3275.44433583801,0.774724910911141
2,1992,Algeria,8383.77024779121,2.96498636282543
5,1992,Angola,5148.25282361678,0.410522931864339
6,1992,Antigua and Barbuda,15497.4647441974,4.0952366180143
7,1992,Argentina,16209.3230774449,3.61928035773818
...,...,...,...,...
5417,2016,Vanuatu,3061.78723983667,0.527000323357166
5419,2016,Vietnam,6767.90249535442,2.05756591685295
5421,2016,West Bank and Gaza,6438.93364028868,0.740606555214825
5423,2016,Zambia,3467.87515599636,0.314182894901441


In [271]:
temp.groupby('country').agg(lambda x: x.isnull().sum()).sort_values(['gdppc']+['co2pc'],ascending=False).head(60)

,year,gdppc,co2pc
country,,,
Afghanistan,0,0,0
Albania,0,0,0
Algeria,0,0,0
Angola,0,0,0
Antigua and Barbuda,0,0,0
Argentina,0,0,0
Armenia,0,0,0
Aruba,0,0,0
Australia,0,0,0


In [272]:
temp.head()

,year,country,gdppc,co2pc
1,1992,Albania,3275.44433583801,0.774724910911141
2,1992,Algeria,8383.77024779121,2.96498636282543
5,1992,Angola,5148.25282361678,0.410522931864339
6,1992,Antigua and Barbuda,15497.4647441974,4.0952366180143
7,1992,Argentina,16209.3230774449,3.61928035773818


In [273]:
temp.columns

Index(['year', 'country', 'gdppc', 'co2pc'], dtype='object')

In [274]:
print(type(temp.gdppc[1]))
print(type(temp.co2pc[1]))

<class 'str'>
<class 'str'>


In [275]:
temp.gdppc=pd.to_numeric(temp.gdppc)
temp.co2pc=pd.to_numeric(temp.co2pc)
temp.year=pd.to_numeric(temp.year)

In [276]:
countries_grouped = temp.groupby("country")

In [277]:
temp["lngdppc"] = countries_grouped["gdppc"].transform(np.log)
temp["lnco2pc"] = countries_grouped["co2pc"].transform(np.log)
temp["d_lngdppc"] = countries_grouped["lngdppc"].transform("diff")
temp["d_lnco2pc"] = countries_grouped["lnco2pc"].transform("diff")

In [278]:
temp=temp.dropna(axis=0,thresh=7)

In [279]:
temp.isnull().sum()

year         0
country      0
gdppc        0
co2pc        0
lngdppc      0
lnco2pc      0
d_lngdppc    0
d_lnco2pc    0
dtype: int64

In [280]:
temp.groupby('country').size().sort_values().head(20)

country
Sint Maarten (Dutch part)     2
Djibouti                      3
Sudan                         4
Curacao                       4
Turks and Caicos Islands      5
Kosovo                        8
Serbia                       10
Cayman Islands               10
Montenegro                   11
Nauru                        12
Timor-Leste                  14
Afghanistan                  14
Sao Tome and Principe        15
Palau                        16
Liberia                      16
Qatar                        16
Libya                        17
Moldova                      21
Maldives                     21
Croatia                      21
dtype: int64

In [286]:
###Dropping contries wiht poor coverage (<16 observations)
countries_to_drop=['Sao Tome and Principe','Sudan','Timor-Leste','Djibouti','Curacao','Cayman Islands',
                   'Sint Maarten (Dutch part)','Kosovo','Montenegro','Nauru','Serbia',
                   'Afghanistan','Turks and Caicos Islands',]
temp=temp.set_index('country').drop(index=countries_to_drop,axis=0)

In [287]:
temp.groupby('country').size().sort_values().head(20)

country
Qatar                     16
Liberia                   16
Palau                     16
Libya                     17
Lithuania                 21
Maldives                  21
Moldova                   21
Croatia                   21
Kuwait                    21
Latvia                    21
France                    22
West Bank and Gaza        22
Bosnia and Herzegovina    22
Italy                     22
Estonia                   23
Cambodia                  23
Madagascar                24
Namibia                   24
Nepal                     24
Netherlands               24
dtype: int64

In [288]:
temp.reset_index(inplace=True)


In [292]:
temp.query("year == 1999 | year == 2000").head(20)

,country,year,gdppc,co2pc,lngdppc,lnco2pc,d_lngdppc,d_lnco2pc
1020,Albania,1999,5492.654212,0.960164,8.611167,-0.040651,0.127576,0.538683
1021,Algeria,1999,8506.345679,3.005719,9.048568,1.100517,0.017336,-0.164704
1022,Angola,1999,4740.669833,0.577083,8.463934,-0.549769,-0.010912,0.192956
1023,Antigua and Barbuda,1999,17551.305968,4.419611,9.772884,1.486052,0.016797,0.014293
1024,Argentina,1999,18981.168384,4.030589,9.851203,1.393912,-0.045574,0.038870
1025,Armenia,1999,3798.671291,0.975802,8.242407,-0.024496,0.038813,-0.103021
1026,Aruba,1999,38910.416818,9.105185,10.569017,2.208844,-0.007301,-0.015070
1027,Australia,1999,37450.358069,17.190298,10.530772,2.844345,0.038066,0.014922
1028,Austria,1999,45139.742923,7.759483,10.717518,2.048916,0.033000,-0.030392
1029,Azerbaijan,1999,3687.661015,3.579835,8.212748,1.275317,0.062614,-0.111721


In [306]:
formula="d_lnco2pc ~ d_lngdppc"

###Pooled regression 2000
ols_pooled = smf.ols("d_lnco2pc ~ d_lngdppc + country", temp.query("year == 2000")).fit(cov_type="HC0")

In [307]:

models = [ols_pooled]
names = ["Pooled regression 1999/2000"]
stargazer = Stargazer(models)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "d_lngdppc": "GDP per capita log change, cumulative coeff.",
        "d_lnco2pc": "CO2 emissions per capita log change, cumulative coeff."
    }
)

stargazer.covariate_order(
    [
        "d_lngdppc",
        "Intercept"
    ]
)

stargazer.custom_columns(names, [1])
HTML(stargazer.render_html())

The per capita CO2 emissions increased by 0.8% more, on average, in 2000 than in 1999 after 1% increase in GDP per capita, controlling for a country.

In [304]:
temp.head()

,country,year,gdppc,co2pc,lngdppc,lnco2pc,d_lngdppc,d_lnco2pc
0,Albania,1993,3610.520633,0.723790,8.191607,-0.323254,0.097398,-0.068006
1,Algeria,1993,8027.376688,2.974812,8.990613,1.090181,-0.043440,0.003309
2,Angola,1993,3788.508107,0.441721,8.239728,-0.817077,-0.306685,0.073247
3,Antigua and Barbuda,1993,15991.764201,4.125424,9.679829,1.417169,0.031397,0.007344
4,Argentina,1993,17312.030499,3.471494,9.759157,1.244585,0.065815,-0.041690


In [ ]:
formula="d_lnco2pc ~ d_lngdppc"

###Pooled regression 1999/2000
ols_pooled = smf.ols("d_lnco2pc ~ d_lngdppc + country", temp.query("year == 1999 | year == 2000")).fit(cov_type="HC0")

In [31]:
###Linear model
from linearmodels import PanelOLS

In [33]:
temp.head()

,,gdppc,co2pc
country,year,,
Albania,1992,3275.44433583801,0.774724910911141
Algeria,1992,8383.77024779121,2.96498636282543
Angola,1992,5148.25282361678,0.410522931864339
Antigua and Barbuda,1992,15497.4647441974,4.0952366180143
Argentina,1992,16209.3230774449,3.61928035773818


In [ ]:
lm = PanelOLS.from_formula(
    "surv ~ imm + TimeEffects + EntityEffects",
    data_balanced,
    weights=data_balanced["avgpop"],
).fit(cov_type="clustered", cluster_entity=True)